In [21]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import time

In [22]:
# Load California Housing dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

In [23]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Scale features (important for regression!)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
print("Data prepared successfully!")
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Data prepared successfully!
Training data shape: (16512, 8)
Test data shape: (4128, 8)


# BASELINE: Original Regression Model

In [26]:
model = Sequential([
    Dense(128, activation="relu", input_shape=(8,)),
    Dense(64, activation="relu"),
    Dense(1, activation="linear")  # 1 output for regression, linear activation
])

C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,473 (37.00 KB)

 Trainable params: 9,473 (37.00 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer="adam",
              loss="mse",          
              metrics=["mae"])     

In [29]:
start_time = time.time()
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)
baseline_time = time.time() - start_time

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.7942 - mae: 0.5729 - val_loss: 0.4240 - val_mae: 0.4554
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3800 - mae: 0.4365 - val_loss: 0.4034 - val_mae: 0.4516
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3555 - mae: 0.4207 - val_loss: 0.4034 - val_mae: 0.4319
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.3385 - mae: 0.4086 - val_loss: 0.3684 - val_mae: 0.4263
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3254 - mae: 0.4003 - val_loss: 0.3381 - val_mae: 0.4013


In [30]:
y_pred = model.predict(X_test, verbose=0)
baseline_mse = mean_squared_error(y_test, y_pred)
baseline_rmse = np.sqrt(baseline_mse)
print(f"Baseline - RMSE: {baseline_rmse:.4f} (${baseline_rmse*100000:.0f}), Time: {baseline_time:.1f}s")

Baseline - RMSE: 0.5719 ($57185), Time: 23.7s


# Tanh vs ReLU Hidden Layers

In [31]:
# Model with Tanh
print("\n--- Testing Tanh Hidden Layers ---")
model_tanh = Sequential([
    Dense(128, activation="tanh", input_shape=(8,)),  
    Dense(64, activation="tanh"),                    
    Dense(1, activation="linear")
])


--- Testing Tanh Hidden Layers ---


C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
model_tanh.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [33]:
start_time = time.time()
model_tanh.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)
tanh_time = time.time() - start_time

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.7435 - mae: 0.6068 - val_loss: 0.5143 - val_mae: 0.5243
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.4646 - mae: 0.4986 - val_loss: 0.5281 - val_mae: 0.5413
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.4511 - mae: 0.4903 - val_loss: 0.4792 - val_mae: 0.4882
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.4263 - mae: 0.4709 - val_loss: 0.4760 - val_mae: 0.4921
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.3972 - mae: 0.4502 - val_loss: 0.4362 - val_mae: 0.4670


In [34]:
y_pred = model_tanh.predict(X_test, verbose=0)
tanh_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Tanh - RMSE: {tanh_rmse:.4f} (${tanh_rmse*100000:.0f}), Time: {tanh_time:.1f}s")

Tanh - RMSE: 0.6323 ($63234), Time: 29.2s


# Different Output Activations

In [35]:
print("\n--- Testing Sigmoid Output ---")
model_sigmoid_out = Sequential([
    Dense(128, activation="relu", input_shape=(8,)),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")  # Changed from linear
])


--- Testing Sigmoid Output ---


C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model_sigmoid_out.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_sigmoid_out.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)
y_pred = model_sigmoid_out.predict(X_test, verbose=0)
sigmoid_out_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Sigmoid Output - RMSE: {sigmoid_out_rmse:.4f} (${sigmoid_out_rmse*100000:.0f})")

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 2.5279 - mae: 1.1698 - val_loss: 2.5372 - val_mae: 1.1610
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2.4803 - mae: 1.1539 - val_loss: 2.5370 - val_mae: 1.1610
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2.4802 - mae: 1.1539 - val_loss: 2.5370 - val_mae: 1.1610
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.4802 - mae: 1.1539 - val_loss: 2.5369 - val_mae: 1.1610
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2.4801 - mae: 1.1538 - val_loss: 2.5368 - val_mae: 1.1608
Sigmoid Output - RMSE: 1.5567 ($155670)


In [37]:
# Model with ReLU Output (for comparison)
print("\n--- Testing ReLU Output ---")
model_relu_out = Sequential([
    Dense(128, activation="relu", input_shape=(8,)),
    Dense(64, activation="relu"),
    Dense(1, activation="relu")  # Changed from linear
])


--- Testing ReLU Output ---


C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
model_relu_out.compile(optimizer="adam", loss="mse", metrics=["mae"])
model_relu_out.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)
y_pred = model_relu_out.predict(X_test, verbose=0)
relu_out_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"ReLU Output - RMSE: {relu_out_rmse:.4f} (${relu_out_rmse*100000:.0f})")

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.7242 - mae: 0.5720 - val_loss: 0.4324 - val_mae: 0.4631
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3834 - mae: 0.4397 - val_loss: 0.4158 - val_mae: 0.4465
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3655 - mae: 0.4272 - val_loss: 0.3940 - val_mae: 0.4524
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3589 - mae: 0.4207 - val_loss: 0.3897 - val_mae: 0.4426
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3315 - mae: 0.4034 - val_loss: 0.3505 - val_mae: 0.4169
ReLU Output - RMSE: 0.5788 ($57876)


# Doubling Hidden Layers

In [39]:
# Model with 4 hidden layers (double the original 2)
print("\n--- Testing 4 Hidden Layers ---")
model_deep = Sequential([
    Dense(128, activation="relu", input_shape=(8,)),
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(64, activation="relu"),  # Added 2 more layers
    Dense(1, activation="linear")
])


--- Testing 4 Hidden Layers ---


C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
model_deep.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,145 (117.75 KB)

 Trainable params: 30,145 (117.75 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model_deep.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [42]:
start_time = time.time()
model_deep.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)
deep_time = time.time() - start_time

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.5479 - mae: 0.5131 - val_loss: 0.4946 - val_mae: 0.4808
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3647 - mae: 0.4253 - val_loss: 0.3913 - val_mae: 0.4319
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3326 - mae: 0.4031 - val_loss: 0.4007 - val_mae: 0.4285
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3155 - mae: 0.3896 - val_loss: 0.3387 - val_mae: 0.4004
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3068 - mae: 0.3832 - val_loss: 0.3355 - val_mae: 0.4096


In [43]:
y_pred = model_deep.predict(X_test, verbose=0)
deep_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Deep Model - RMSE: {deep_rmse:.4f} (${deep_rmse*100000:.0f}), Time: {deep_time:.1f}s")

Deep Model - RMSE: 0.5636 ($56357), Time: 17.1s


# Different Optimizers

In [44]:
# SGD Optimizer
print("\n--- Testing SGD Optimizer ---")
model_sgd = Sequential([
    Dense(128, activation="relu", input_shape=(8,)),
    Dense(64, activation="relu"),
    Dense(1, activation="linear")
])


--- Testing SGD Optimizer ---


C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
model_sgd.compile(optimizer="sgd", loss="mse", metrics=["mae"])

In [46]:
start_time = time.time()
model_sgd.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)
sgd_time = time.time() - start_time

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.6959 - mae: 0.5787 - val_loss: 0.4841 - val_mae: 0.4901
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4319 - mae: 0.4688 - val_loss: 0.4505 - val_mae: 0.4900
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4052 - mae: 0.4517 - val_loss: 0.4237 - val_mae: 0.4589
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3864 - mae: 0.4400 - val_loss: 0.4114 - val_mae: 0.4508
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3728 - mae: 0.4321 - val_loss: 0.4104 - val_mae: 0.4418


In [47]:
y_pred = model_sgd.predict(X_test, verbose=0)
sgd_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"SGD - RMSE: {sgd_rmse:.4f} (${sgd_rmse*100000:.0f}), Time: {sgd_time:.1f}s")

SGD - RMSE: 0.6274 ($62737), Time: 12.4s


In [48]:
# RMSprop Optimizer
print("\n--- Testing RMSprop Optimizer ---")
model_rmsprop = Sequential([
    Dense(128, activation="relu", input_shape=(8,)),
    Dense(64, activation="relu"),
    Dense(1, activation="linear")
])


--- Testing RMSprop Optimizer ---


C:\Users\bbuser\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
model_rmsprop.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

In [50]:
start_time = time.time()
model_rmsprop.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)
rmsprop_time = time.time() - start_time

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.6583 - mae: 0.5469 - val_loss: 0.4319 - val_mae: 0.4665
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3925 - mae: 0.4415 - val_loss: 0.4437 - val_mae: 0.4963
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3764 - mae: 0.4233 - val_loss: 0.3700 - val_mae: 0.4196
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3507 - mae: 0.4112 - val_loss: 0.3667 - val_mae: 0.4340
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3350 - mae: 0.4028 - val_loss: 0.3724 - val_mae: 0.4069


In [51]:
y_pred = model_rmsprop.predict(X_test, verbose=0)
rmsprop_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSprop - RMSE: {rmsprop_rmse:.4f} (${rmsprop_rmse*100000:.0f}), Time: {rmsprop_time:.1f}s")

RMSprop - RMSE: 0.6030 ($60299), Time: 14.1s
